In [7]:
#Importing all the necessary packages
import pandas as pd
import mysql.connector
from datetime import datetime

#creating an connection with DB
#mydb = MySQLdb.connect(host='localhost',user='nishtha',passwd='Nisnik@123',db='plm')
#mydb ='mysql+mysqlconnector://{user}:{password}@{server}/{database}'.format(user='nishtha', password='Nisnik@123', server='localhost', database='plm')
mydb = mysql.connector.connect(
  host="localhost",
  user="nishtha",
  password="Nisnik@123",
  database="plm"
)
cursor = mydb.cursor()

# Loading Data
data = pd.read_csv ('user_symptom.csv')
df_symptom = pd.DataFrame(data, columns= ['user_id','symptom_name','symptom_severity_score'])

data = pd.read_csv ('user_onset_date.csv')
df_onset_date = pd.DataFrame(data, columns= ['user_id','first_definitive_diagnosis_date'])

data = pd.read_csv ('user_condition.csv')
df_condition = pd.DataFrame(data, columns= ['user_id','condition_name'])

data = pd.read_csv ('user_ALSFRS_score.csv')
df_ALSFRS_score = pd.DataFrame(data, columns= ['user_id','report_date','score'])

#Data Cleaning
df_symptom.symptom_name = df_symptom.symptom_name[0:50]
df_symptom = df_symptom.dropna()
df_onset_date = df_onset_date.dropna()
df_condition = df_condition.dropna()
df_ALSFRS_score = df_ALSFRS_score.dropna()

#Creating table for each file in MySQL
cursor.execute('CREATE TABLE user_symptom (user_id int, symptom_name nvarchar(50), symptom_severity_score int)')
cursor.execute('CREATE TABLE user_onset_date (user_id int, first_definitive_diagnosis_date date)')
cursor.execute('CREATE TABLE user_condition (user_id int, condition_name nvarchar(50))')
cursor.execute('CREATE TABLE user_ALSFRS_score (user_id int, report_date date, score int)')

#Inserting values to the tables created in the MySQL Database
for row in df_symptom.itertuples():
    cursor.execute("INSERT INTO user_symptom (user_id, symptom_name, symptom_severity_score) VALUES (%s, %s, %s)", (row.user_id, row.symptom_name, row.symptom_severity_score))

for row in df_onset_date.itertuples():
    res = cursor.execute("""INSERT INTO `user_onset_date` (`user_id`, `first_definitive_diagnosis_date`) VALUES (%(int)s, %(date)s)""", {'int': row.user_id, 'date': datetime.strptime(row.first_definitive_diagnosis_date,'%Y-%m-%d')})
    
for row in df_condition.itertuples():
    cursor.execute("INSERT INTO user_condition (user_id, condition_name) VALUES (%(int)s, %(str)s)", {'int': row.user_id, 'str': row.condition_name})

for row in df_ALSFRS_score.itertuples():
    cursor.execute("INSERT INTO user_ALSFRS_score (user_id, report_date, score) VALUES (%s, %s, %s)", (row.user_id, row.report_date, row.score))
    
#close the connection to the database
mydb.commit()
cursor.close()
print("Done")

Done


In [3]:
pip install MySQLClient

Note: you may need to restart the kernel to use updated packages.
